In [109]:
import pandas as pd
import os
import re

# funcion que busca la posicion de la palabra
def finder(x):
    for i in range(len(lineas_pro)):
        try:
            return i,lineas_pro[i].index(x)
        except ValueError:
            pass    

In [508]:
data_path = r"D:\Users\WS-012\Desktop\P_Colmedica\estructura\Join_BD_prueba\\" 
df = pd.read_excel("output_bienestar.xlsx") 
df2 = pd.DataFrame()

In [170]:
df.head()

,id_paciente,Prestador,ruta_pdf,nombre_pdf,tot_paginas,pagina,texto_pag
0,1093820,Bienestar,D:\Users\WS-012\Desktop\BD_PDF\Join_BD_prueba\...,CC_1093820.pdf,236,1,['RECORD CLINICO\nHISTORIA CLINICA\n\nGenero\n...
1,1093820,Bienestar,D:\Users\WS-012\Desktop\BD_PDF\Join_BD_prueba\...,CC_1093820.pdf,236,2,['RECORD CLINICO\nHISTORIA CLINICA\n\nHIDROCLO...
2,1093820,Bienestar,D:\Users\WS-012\Desktop\BD_PDF\Join_BD_prueba\...,CC_1093820.pdf,236,3,['RECORD CLINICO\nHISTORIA CLINICA\n\nNO FUMA\...
3,1093820,Bienestar,D:\Users\WS-012\Desktop\BD_PDF\Join_BD_prueba\...,CC_1093820.pdf,236,4,['RECORD CLINICO\nHISTORIA CLINICA\n\nOtros an...
4,1093820,Bienestar,D:\Users\WS-012\Desktop\BD_PDF\Join_BD_prueba\...,CC_1093820.pdf,236,5,['RECORD CLINICO\nHISTORIA CLINICA\n\nTipo de ...


In [112]:
df.dtypes

id_paciente     int64
Prestador      object
ruta_pdf       object
nombre_pdf     object
tot_paginas     int64
pagina          int64
texto_pag      object
dtype: object

In [509]:
for infolder_pacientes in os.listdir(data_path):
    tipo = os.path.splitext(infolder_pacientes)[1]
    
    # si el archivo dentro del paciente no es un pdf 
    if os.path.splitext(infolder_pacientes)[1] != ".pdf":
        folder_prestador = infolder_pacientes                                                  # ---------folder_paciente        
        
        # leo los archivos dentro del folder del paciente
        for fold_paciente in os.listdir(data_path+folder_prestador+"/"):
            folder_paciente = fold_paciente                                                 # ---------folder_prestador
            
            if folder_prestador == "Bienestar":
                # leo los archivos dentro del folder del prestador
                for file_prestador in os.listdir(data_path+folder_prestador+"/"+folder_paciente+"/"):

                    # si el archivo es un pdf
                    if os.path.splitext(file_prestador)[1] == ".pdf" or os.path.splitext(file_prestador)[1] == ".PDF" :                        
                        nombre_pdf = file_prestador 

                        paciente = os.path.splitext(nombre_pdf)[0]
                        ruta_pdf = r"D:\Users\WS-012\Desktop\P_Colmedica\estructura\data_path"+"\\"+folder_paciente+"\\"+folder_prestador+"\\"+nombre_pdf  # ----------- ruta pdf
                        id_paciente = paciente.split("_")[1]                                        # --------------- id

                        # filtro por id, prestador y nombre del pdf
                        filtro = df[(df["id_paciente"] == int(id_paciente)) &
                                    (df["Prestador"] == folder_prestador) &
                                    (df["nombre_pdf"] ==  nombre_pdf)]

                        texto_pag_filtro = filtro["texto_pag"]
                        
                        union = ""
                        for pagina in texto_pag_filtro:
                            union = union + pagina
                            union = union.lower()

                        new = union.replace('[\'',' ')
                        new = new.replace('\']',' ')
                        new = new.replace('\\n',',')
                        new_n = new.split(sep=',')

                        lineas = [linea.split() for linea in new_n]

                        lineas_unidas = []
                        for i in range(len(lineas)):
                            lineas_unidas += lineas[i]

                        lineas_pro = [lineas_unidas]
                        
                        

                        # Datos personales ------------------------------
                        v1 = lineas_pro[finder('datos')[0]][finder('datos')[1]:finder('datos')[1]+100] 
                        frase_1 = " ".join(v1)
                        s1 = [s for s in re.findall(r'(?=datos\sde\sidentificación)(.+)(?=antecedentes)',frase_1)]
                        
                        d1 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': "Datos personales" , 
                             'texto_secciones': s1[0],
                             'fecha_consulta': "No aplica",
                              'seccciones_consulta': "No aplica",
                              'texto_secciones_consulta': "No aplica"
                            }

                        df2 = df2.append(d1,ignore_index=True)

                        # Antecedentes ------------------------------
                        v2 = lineas_pro[finder('antecedentes')[0]][finder('antecedentes')[1]:finder('antecedentes')[1]+10000] 
                        frase_2 = " ".join(v2)
                        s2 = [s for s in re.findall(r'(?=antecedentes\s\(inicio\))(.+)(?=consultas\s\(inicio\))',frase_2)]
                        
                        d2 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': "Antecedentes" , 
                             'texto_secciones': s2[0],
                              'fecha_consulta': "No aplica",
                              'seccciones_consulta': "No aplica",
                              'texto_secciones_consulta': "No aplica"
                            }

                        df2 = df2.append(d2,ignore_index=True)

                        # Total consultas ------------------------------
                        v3 = lineas_pro[finder('consultas')[0]][finder('consultas')[1]:finder('consultas')[1]+1000000000000] 
                        frase_3 = " ".join(v3)
                        s3 = [s for s in re.findall(r'(?=consultas\s\(inicio\))(.+)(?=consultas\s\(fin\))',frase_3)]
                        
                        d3 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': "Total consultas" , 
                             'texto_secciones': s3[0],
                              'fecha_consulta': "No aplica",
                              'seccciones_consulta': "No aplica",
                              'texto_secciones_consulta': "No aplica"
                            }

                        df2 = df2.append(d3,ignore_index=True)

                        # Consultas ------------------------------
                        v4 = lineas_pro[finder('consultas')[0]][finder('consultas')[1]:finder('consultas')[1]+1000000000000] 
                        frase_4 = " ".join(v4)
                        s4 = [s for s in re.split(r'(?=consulta\s\-\s\#\sinterno:\s[0-9]{1,})',frase_4)]
                        #s4[1]

                        s4_1 = [s for s in re.findall(r'(?:consulta\s\-\s\#\sinterno:\s[0-9]{1,})', str(s4))]
                        #nombre_consulta[0]
                        
                        for inconsulta in range(1,len(s4)-1):
                            consulta = s4[inconsulta]
                            seccion_consulta = s4_1[inconsulta-1]
    
                            fecha_consulta = [s for s in re.findall(r'\d{4}\-\d{2}\-\d{2}\s\d{2}\:\d{2}\:\d{2}',consulta)]
                            if fecha_consulta == []:
                                fecha_consulta = ["No Presenta fecha"]

                            # Sintesis Historia Clínica
                            s4_s1 = [s for s in re.findall(r'(?:consulta\s\-\s\#\sinterno:\s[0-9]{1,})(.+)(?:pa\sdia)', str(s4[inconsulta]))]
                            if s4_s1 == []:
                                s4_s1 = ["No aplica"]
                            d4 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': seccion_consulta , 
                             'texto_secciones': consulta,
                             'fecha_consulta': fecha_consulta[0],
                              'seccciones_consulta': "Sintesis Historia Clínica" ,
                              'texto_secciones_consulta': s4_s1[0]
                            }
                            df2 = df2.append(d4,ignore_index=True)

                            # Examen fisico
                            s4_s2 = [s for s in re.findall(r'(?:pa\sdia)(.+)(?:abdomen\s)', str(s4[inconsulta]))]
                            if s4_s2 == []:
                                s4_s2 = ["No aplica"]
                            d4 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': seccion_consulta , 
                             'texto_secciones': consulta,
                             'fecha_consulta': fecha_consulta[0],
                              'seccciones_consulta': "Examen fisico" ,
                              'texto_secciones_consulta': s4_s2[0]
                            }
                            df2 = df2.append(d4,ignore_index=True)

                            # resumen y comentarios
                            s4_s3 = [s for s in re.findall(r'(?:abdomen\s)(.+)(?:diagnostico|dX)', str(s4[inconsulta]))]
                            if s4_s3 == []:
                                s4_s3 = ["No aplica"]
                            d4 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': seccion_consulta , 
                             'texto_secciones': consulta,
                             'fecha_consulta': fecha_consulta[0],
                              'seccciones_consulta': "Resumen y comentarios" ,
                              'texto_secciones_consulta': s4_s3[0]
                            }
                            df2 = df2.append(d4,ignore_index=True)

                            # Diagnostico, si no encuentra la paalabra conducta toma todo
                            s4_s4 = [s for s in re.findall(r'(?=dx\sppal:)(.+)(?:conducta)', str(s4[inconsulta]))]
                            if s4_s4 == []:
                                s4_s4 = [s for s in re.findall(r'(?=dx\sppal:)(.+)', str(s4[inconsulta]))]
                                if s4_s4 == []:
                                    s4_s4 = ["No aplica"]

                            d4 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': seccion_consulta , 
                             'texto_secciones': consulta,
                             'fecha_consulta': fecha_consulta[0],
                              'seccciones_consulta': "Diagnostico" ,
                              'texto_secciones_consulta': s4_s4[0]
                            }
                            df2 = df2.append(d4,ignore_index=True)

                            # Conducta
                            s4_s5 = [s for s in re.findall(r'(?=conducta)(.+)', str(s4[inconsulta]))]
                            if s4_s5 == []:
                                s4_s5 = ["No aplica"]

                            d4 = {'ruta_pdf': ruta_pdf,
                             'nombre_pdf':nombre_pdf,
                             'Prestador': folder_prestador,
                             'Seccion': seccion_consulta , 
                             'texto_secciones': consulta,
                             'fecha_consulta': fecha_consulta[0],
                              'seccciones_consulta': "Conducta" ,
                              'texto_secciones_consulta': s4_s5[0]
                            }
                            df2 = df2.append(d4,ignore_index=True)
                            
                            
                            
                        

                        # Programas especiales y enfermedades cronicas ------------------------------
                        
                        #v5 = lineas_pro[finder('Programas')[0]][finder('Programas')[1]:finder('Programas')[1]+10000] 
                        #frase_5 = " ".join(v5)
                        #s5 = [s for s in re.findall(r'(?=Programas\sEspeciales\s\-\sEnfermedades\sCronicas\s\(Inicio\))(.+)(?=Programas\sEspeciales\s\-\sEnfermedades\sCronicas\s\(Fin\))',frase_5)]


                        # Notas asistenciales ------------------------------
                        #v6 = lineas_pro[finder('Notas')[0]][finder('Notas')[1]:finder('Notas')[1]+10000] 
                        #frase_6 = " ".join(v6)
                        #s6 = [s for s in re.findall(r'(?=Notas\sAsistenciales\s\(Inicio\))(.+)(?=Notas\sAsistenciales\s\(Fin\))',frase_6)]

                        # -------------------------------------
                        
                        #df.to_excel(excel_writer = r"D:\Users\WS-012\Desktop\BD_PDF\BD_Excel\output.xlsx" ,index = False, engine='xlsxwriter') 


D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d1,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d2,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d3,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:180: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:180: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

...

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:180: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d1,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d2,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d3,ignore_index=True)
D:\Users\WS-012\AppData\Local\

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:180: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:180: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:180: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Local\Temp\1\ipykernel_9580\51379155.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(d4,ignore_index=True)
D:\Users\WS-012\AppData\Loca

In [510]:
df2

,ruta_pdf,nombre_pdf,Prestador,Seccion,texto_secciones,fecha_consulta,seccciones_consulta,texto_secciones_consulta
0,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CC_1093820.pdf,Bienestar,Datos personales,datos de identificación identificación cc-1093...,No aplica,No aplica,No aplica
1,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CC_1093820.pdf,Bienestar,Antecedentes,antecedentes (inicio) patológicos hipertension...,No aplica,No aplica,No aplica
2,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CC_1093820.pdf,Bienestar,Total consultas,consultas (inicio) angie tatiana moyano buitra...,No aplica,No aplica,No aplica
3,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CC_1093820.pdf,Bienestar,consulta - # interno: 3000247777,consulta - # interno: 3000247777 profesional: ...,2022-07-27 16:09:00,Sintesis Historia Clínica,profesional: especialidad: responsable nombre...
4,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CC_1093820.pdf,Bienestar,consulta - # interno: 3000247777,consulta - # interno: 3000247777 profesional: ...,2022-07-27 16:09:00,Examen fisico,temp fc fr sat o2 glucom peso(kg) talla(cm) i...
...,...,...,...,...,...,...,...,...
2605,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CE_470920.pdf,Bienestar,consulta - # interno: 2021437408,consulta - # interno: 2021437408 profesional: ...,2021-08-09 12:23:00,Sintesis Historia Clínica,profesional: especialidad: brayan styven cani...
2606,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CE_470920.pdf,Bienestar,consulta - # interno: 2021437408,consulta - # interno: 2021437408 profesional: ...,2021-08-09 12:23:00,Examen fisico,temp fc fr sat o2 glucom peso(kg) talla(cm) i...
2607,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CE_470920.pdf,Bienestar,consulta - # interno: 2021437408,consulta - # interno: 2021437408 profesional: ...,2021-08-09 12:23:00,Resumen y comentarios,No aplica
2608,D:\Users\WS-012\Desktop\P_Colmedica\estructura...,CE_470920.pdf,Bienestar,consulta - # interno: 2021437408,consulta - # interno: 2021437408 profesional: ...,2021-08-09 12:23:00,Diagnostico,dx ppal: z538 - procedimiento no realizado por...


In [511]:
df2.to_excel(excel_writer = r"D:\Users\WS-012\Desktop\BD_PDF\BD_Excel\output_rasa_in_sep.xlsx" ,index = False, engine='xlsxwriter')

seccciones_consulta
    1. Sintesis Historia Clínica
    2. Examen fisico
    3. resumen y comentarios
    4. diagnostico
    5. conducta